In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
sys.path.append("/home/timssh/ML/TAGGING/CLS/classification")
from os.path import join
from scripts.utils import parse_meta, get_meta_id
from train.model import InferDataset
from train.augmentation import PreProcess, DataAugmentation

Aug = DataAugmentation().eval()
Pre = PreProcess(gray=False, vflip=False, arch="eff")

In [2]:
GROUP = ['hair_color']
PATH2TEST = '/home/timssh/ML/TAGGING/test'

old_train = {
    "models": {
        # "body_type": "/home/timssh/ML/TAGGING/source_old/wandb/body_type/version_0_train_eff_44_0.01/checkpoints/epoch=52-step=3604.pt",
        # "tits_size": "/home/timssh/ML/TAGGING/source_old/wandb/tits_size/version_0_train_eff_32_0.01/checkpoints/epoch=90-step=11193.pt",
        # "sex_positions": "/home/timssh/ML/TAGGING/source_old/wandb/sex_positions/version_0_train_eff_44_0.01/checkpoints/epoch=47-step=18384.pt",
        "hair_color": "/home/timssh/ML/TAGGING/source_old/wandb/hair_color/version_5_train_eff_44_0.01/checkpoints/epoch=50-step=4386.pt",
        # "hair_type": "/home/timssh/ML/TAGGING/source_old/wandb/hair_type/version_1_val_eff_44_0.01/checkpoints/epoch=76-step=2156.pt",
    },
    "source": "/home/timssh/ML/TAGGING/source_old",
}
valid_train = {
    "models": {
        # "sex_positions": "/home/timssh/ML/TAGGING/source_valid/wandb/sex_positions/version_0_train_eff_32_0.001/checkpoints/epoch=69-step=119560.pt",
        # "hair_type": "/home/timssh/ML/TAGGING/source_valid/wandb/hair_type/version_0_train_eff_32_0.001/checkpoints/epoch=138-step=137332.pt"
        # "tits_size": "/home/timssh/ML/TAGGING/source_valid/wandb/tits_size/version_1_train_eff_36_0.01/checkpoints/epoch=65-step=71016.pt",
        "hair_color": "/home/timssh/ML/TAGGING/source_valid/wandb/hair_color/version_0_train_eff_32_0.001/checkpoints/epoch=121-step=170068.pt",
    },
    "source": "/home/timssh/ML/TAGGING/source_valid",
}
path_meta = '/home/timssh/ML/TAGGING/test/tits_size/1/mofos-solo-porn-493'

In [3]:
# GROUPS = old_train['models'].keys()
GROUPS = ["tits_size"]


if path_meta:
    meta_real = {path_meta : get_meta_id(path_meta + '/meta.json')}

meta_old = parse_meta(old_train['models'], InferDataset, Aug, Pre, old_train['source'], GROUPS, PATH2TEST)
meta_valid = parse_meta(valid_train['models'], InferDataset, Aug, Pre, valid_train['source'], GROUPS, PATH2TEST)

tits_size
/home/timssh/ML/TAGGING/test/tits_size/1/mofos-solo-porn-493
tits_size
/home/timssh/ML/TAGGING/test/tits_size/1/mofos-solo-porn-493


In [4]:
def find_diff(meta_1, meta_2):
    diff_list = []
    for picset in meta_1.keys():
        for id in meta_1[picset]:
            train_1 = meta_1[picset][id]['trained']
            train_2 = meta_2[picset][id]['trained']
            for group_ in GROUP:
                item_1 = list(filter(lambda x: x['group'] == group_, train_1))
                if len(item_1) > 1:
                    item_1 = [item_1[-1]]
                elif len(item_1) == 0:
                    item_1.append({'group': group_, 'category': []})
                item_2 = list(filter(lambda x: x['group'] == group_, train_2))
                if len(item_2) > 1:
                    item_2 = [item_2[-1]]
                elif len(item_2) == 0:
                    item_2.append({'group': group_, 'category': []})
                if item_1 != item_2:
                    diff_list.append({'path': join(picset,'picture', meta_2[picset][id]['origin']['filename']), 'model_1': item_1, 'model_2': item_2})
    return diff_list

def show_res(path, model_1, model_2):
    from PIL import Image
    import matplotlib.pyplot as plt
    img = Image.open(path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    print('model_1', model_1)
    print('model_2', model_2)


In [5]:
ret = find_diff(meta_valid, meta_old)
print(len(ret))
it = iter(ret)
ret[::10]

66


[{'path': '/home/timssh/ML/TAGGING/test/tits_size/1/mofos-solo-porn-493/picture/07db4ae7aa093215ad231c959c553bb2.jpeg',
  'model_1': [{'group': 'hair_color', 'category': ['black hair']}],
  'model_2': [{'group': 'hair_color', 'category': ['brown hair']}]},
 {'path': '/home/timssh/ML/TAGGING/test/tits_size/1/mofos-solo-porn-493/picture/129380a67b5e5d6e923dca5f06eacf4f.jpeg',
  'model_1': [{'group': 'hair_color', 'category': ['brown hair']}],
  'model_2': [{'group': 'hair_color', 'category': ['black hair']}]},
 {'path': '/home/timssh/ML/TAGGING/test/tits_size/1/mofos-solo-porn-493/picture/b29c9401132ccb64e33dd45f821a7ced.jpeg',
  'model_1': [{'group': 'hair_color', 'category': ['black hair']}],
  'model_2': [{'group': 'hair_color', 'category': ['brown hair']}]},
 {'path': '/home/timssh/ML/TAGGING/test/tits_size/1/mofos-solo-porn-493/picture/d696e01626b240042bea1ef44ab51b59.jpeg',
  'model_1': [{'group': 'hair_color', 'category': ['brown hair']}],
  'model_2': [{'group': 'hair_color', 'ca

In [7]:
from IPython.display import display, clear_output
from ipywidgets import Button, HBox, Text
from IPython.display import Image

data = ret
list_of_buttons = []

def display_image(image_path):
    display(Image(filename=image_path))

def on_button_click(b):
    global current_index
    clear_output()
    print(f"Correct answer: {b.description}")
    current_index += 1
    if current_index < len(data):
        display_next_image()

def item_and_click(item):
    def on_button_click(b):
        list_of_buttons.append((b.description, item['path']))
        global current_index
        clear_output()
        print(f"Correct answer: {b.description}")
        current_index += 1
        if current_index < len(data):
            display_next_image()
    return on_button_click

def on_custom_button_click(b):
    global current_index
    clear_output()
    print(f"Correct answer: {custom_text.value}")
    current_index += 1
    if current_index < len(data):
        display_next_image()

def text_and_click(item, custom_text):
    def on_custom_button_click(b):
        global current_index
        clear_output()
        list_of_buttons.append(("Custom: " + custom_text.value, item['path']))
        print(f"Correct answer: {custom_text.value}")
        current_index += 1
        if current_index < len(data):
            display_next_image()
    return on_custom_button_click

def display_next_image():
    global current_index
    item = data[current_index]
    display_image(item['path'])
    
    model_1_output = item['model_1'][0]['category']
    model_2_output = item['model_2'][0]['category']
    
    model_1_button = Button(description=f"Model 1: {model_1_output}")
    model_2_button = Button(description=f"Model 2: {model_2_output}")
    
    # model_1_button.on_click(on_button_click)
    # model_2_button.on_click(on_button_click)
    model_1_button.on_click(item_and_click(item))
    model_2_button.on_click(item_and_click(item))
    
    custom_text = Text(value="")
    custom_button = Button(description="Custom")
    
    custom_button.on_click(on_custom_button_click)
    custom_button.on_click(text_and_click(item, custom_text))
    
    # display(HBox([model_1_button, model_2_button, custom_text, custom_button]))
    display(HBox([model_1_button, model_2_button, custom_button]))

current_index = 0
display_next_image()


Correct answer: Model 2: ['brown hair']


In [8]:
model_1_count = 0
model_2_count = 0
custom_count = 0
for item in list_of_buttons:
    if item[0][:7] == "Model 1":
        model_1_count += 1
    elif item[0][:7] == "Model 2":
        model_2_count += 1
    else:
        custom_count += 1

print('model_1:', model_1_count, '\n', 'model_2:', model_2_count, '\n', 'custom:', custom_count)

model_1: 30 
 model_2: 36 
 custom: 0


In [8]:
import shutil
import os

# define the source directory and destination directory
dst_dir = "/home/timssh/ML/TAGGING/tested_bad/" + GROUP[0]
os.makedirs(dst_dir, exist_ok=True)
# get the list of file paths
# iterate over the file paths and copy each file to the destination directory
for file_path in list_of_buttons:
    if 'Model 1' not in file_path[0]:
        shutil.copy(file_path[1], dst_dir + '/' + file_path[1].split('/')[-1])
    